# The cover for my Ph.D. thesis

In [ ]:
%matplotlib inline

In [ ]:
from thesis_cover import load_learner, generate_cover

In [ ]:
from glob import glob
fnames = glob("data/*/*pickle")
fname = fnames[0]
learner = load_learner(fname)
generate_cover(learner, None)